In [1]:
from typing import Annotated, Sequence, TypedDict
from dotenv import load_dotenv

load_dotenv()

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage , ToolMessage, SystemMessage, BaseMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain.chat_models import init_chat_model

In [ ]:

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    document_content: str


graph_builder = StateGraph(State)




document_content = ""

@tool
def update(content: str) -> str:
    """Update the document content"""
    global document_content
    document_content = content
    return f"Document content updated to: {content}"

@tool
def save(filename: str) -> str:
    """Save the document content"""
    global document_content
    with open(filename, "w") as f:
        f.write(document_content)
    return f"Document content saved to {filename}"

tools = [update, save]

llm = init_chat_model("openai:gpt-4o-mini").bind_tools(tools)

def agent(state: State):
    system_message = SystemMessage(content=f"You are a helpful assistant that can update the document content and save it to a file. remember to save the document after each update. and show the document content after each update. {state['document_content']}")
    
    if not state["messages"]:
        user_input = "I'm ready to help you update a document. What would you like to create?"
        user_message = HumanMessage(content=user_input)
    else:
        user_input = input("\n📝 What would you like to do with the document? ")
        print(f"\n👤 USER: {user_input}")
        user_message = HumanMessage(content=user_input)

    all_messages = [system_message] + list(state["messages"]) + [user_message]
    
    response = llm.invoke(all_messages)
    
    if hasattr(response, "tool_calls"):
        print(response.tool_calls)
    
    return {"messages": list(all_messages) + [user_message, response]}

def should_continue(state: State):
    messages = state["messages"]
    
    if not messages:
        return 'continue'
    
    for message in reversed(messages):
        if(isinstance(message, ToolMessage) and 'saved' in message.content.lower() and 'document' in message.content.lower()):
            return 'end'
        
    return 'continue'
        
